## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
# Hyper-parameters
epochs = 5
classes = 10
batch_size = 100
learning_rate = 0.001

## Download the MNIST dataset

In [4]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Convolutional Neural Network 

In [6]:
class ConvNet(nn.Module):

    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(classes).to(device)

## Define Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train the model

In [8]:
total_step = len(train_loader)
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss {:.4f}'
            .format(epoch+1, epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss 0.1342
Epoch [1/5], Step [200/600], Loss 0.0935
Epoch [1/5], Step [300/600], Loss 0.0842
Epoch [1/5], Step [400/600], Loss 0.2376
Epoch [1/5], Step [500/600], Loss 0.1142
Epoch [1/5], Step [600/600], Loss 0.0196
Epoch [2/5], Step [100/600], Loss 0.0940
Epoch [2/5], Step [200/600], Loss 0.0113
Epoch [2/5], Step [300/600], Loss 0.0352
Epoch [2/5], Step [400/600], Loss 0.0239
Epoch [2/5], Step [500/600], Loss 0.0047
Epoch [2/5], Step [600/600], Loss 0.0441
Epoch [3/5], Step [100/600], Loss 0.0578
Epoch [3/5], Step [200/600], Loss 0.0095
Epoch [3/5], Step [300/600], Loss 0.0193
Epoch [3/5], Step [400/600], Loss 0.0529
Epoch [3/5], Step [500/600], Loss 0.0215
Epoch [3/5], Step [600/600], Loss 0.0489
Epoch [4/5], Step [100/600], Loss 0.0299
Epoch [4/5], Step [200/600], Loss 0.0464
Epoch [4/5], Step [300/600], Loss 0.1410
Epoch [4/5], Step [400/600], Loss 0.0169
Epoch [4/5], Step [500/600], Loss 0.0358
Epoch [4/5], Step [600/600], Loss 0.0163
Epoch [5/5], Ste

## Test the model

In [9]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test accuracy of the model on the 10000 test images: {} %'
    .format(100 * correct / total)) 

Test accuracy of the model on the 10000 test images: 99.19 %


## Save the model

In [10]:
torch.save(model.state_dict(), 'model.ckpt')